In [1]:
!pip install ydata_profiling;pip install pycaret;pip install country_converter;pip install optuna;

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.1/400.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.8/687.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 15.5 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=079001892256f58828141909888a94f96cad8a4ec8f75117db16e8c4e71a9570
  Stored in directory: /root/.cache/pip/wheels/8d/55/1a/19cd535375ed1ede0c996405ebffe34b196d78e2d9545723a2
Successfully built htmlmin
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import pandas as pd
from ydata_profiling import ProfileReport
from pycaret.regression import *
from collections import Counter
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import TargetEncoder
import lightgbm as lgb
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import optuna
from sklearn.preprocessing import TargetEncoder
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
!unzip /content/predykcja-analiza-cen-aut.zip

Archive:  /content/predykcja-analiza-cen-aut.zip
  inflating: sales_ads_test.csv      
  inflating: sales_ads_train.csv     
  inflating: sample_submission.csv   
  inflating: synthetic_training_data_mostlyai_pl.csv  
  inflating: synthetic_training_data_sdv_pl.csv  


In [2]:
df = pd.read_csv("/content/sales_ads_train.csv",index_col=0)

In [3]:
pd.set_option('display.max_columns', 25)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Cena                        135397 non-null  int64  
 1   Waluta                      132021 non-null  object 
 2   Stan                        132075 non-null  object 
 3   Marka_pojazdu               132046 non-null  object 
 4   Model_pojazdu               132088 non-null  object 
 5   Wersja_pojazdu              87336 non-null   object 
 6   Generacja_pojazdu           93737 non-null   object 
 7   Rok_produkcji               131990 non-null  float64
 8   Przebieg_km                 131394 non-null  float64
 9   Moc_KM                      131664 non-null  float64
 10  Pojemnosc_cm3               130711 non-null  float64
 11  Rodzaj_paliwa               131987 non-null  object 
 12  Emisja_CO2                  59497 non-null   float64
 13  Naped              

In [5]:
df.head()

,Cena,Waluta,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Generacja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,Pojemnosc_cm3,Rodzaj_paliwa,Emisja_CO2,Naped,Skrzynia_biegow,Typ_nadwozia,Liczba_drzwi,Kolor,Kraj_pochodzenia,Pierwszy_wlasciciel,Data_pierwszej_rejestracji,Data_publikacji_oferty,Lokalizacja_oferty,Wyposazenie
ID,,,,,,,,,,,,,,,,,,,,,,,,
1,13900,PLN,Used,Renault,Grand Espace,Gr 2.0T 16V Expression,NaN,2005.0,213000.0,170.0,1998.0,Gasoline,232.0,Front wheels,Manual,minivan,5.0,blue,NaN,NaN,NaN,28/04/2021,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...","['ABS', 'Electric front windows', 'Drivers air..."
2,25900,PLN,Used,Renault,Megane,1.6 16V 110,III (2008-2016),2010.0,117089.0,110.0,1598.0,Gasoline,159.0,Front wheels,Manual,station_wagon,5.0,silver,NaN,NaN,16/06/2010,04/05/2021,"ul. Wiosenna 8 - 41-407 Imielin, Centrum (Polska)","['ABS', 'Electric front windows', 'Drivers air..."
3,35900,PLN,Used,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,C (2011-2019),2015.0,115600.0,136.0,1598.0,NaN,NaN,NaN,Manual,minivan,5.0,white,Denmark,NaN,NaN,03/05/2021,"Sianów, koszaliński, Zachodniopomorskie","['ABS', 'Electric front windows', 'Passengers ..."
4,5999,PLN,Used,Ford,Focus,1.6 TDCi FX Silver / Silver X,Mk2 (2004-2011),2007.0,218000.0,90.0,1560.0,Diesel,118.0,Front wheels,Manual,compact,5.0,blue,NaN,NaN,27/11/2007,02/05/2021,"Gdańsk, Pomorskie, Przymorze Wielkie","['ABS', 'Electric front windows', 'Drivers air..."
5,44800,PLN,Used,Toyota,Avensis,1.8,III (2009-),2013.0,NaN,NaN,1798.0,Gasoline,152.0,Front wheels,Manual,NaN,4.0,other,Poland,Yes,20/05/2013,02/05/2021,"Świdnik, świdnicki, Lubelskie","['ABS', 'Electric front windows', 'Drivers air..."


Po wstępnym zapoznaniu się z danymi celem dokonania benchmarku zostaną wytrenowane modele na nieprzekszałconym zbiorze danych za pomocą bibioteki pycaret

In [6]:
s = setup(df.drop(columns=["Waluta","Data_pierwszej_rejestracji","Data_publikacji_oferty"]), target = 'Cena', session_id = 123, fold = 5)

,Description,Value
0,Session id,123
1,Target,Cena
2,Target type,Regression
3,Original data shape,"(135397, 21)"
4,Transformed data shape,"(135397, 53)"
5,Transformed train set shape,"(94777, 53)"
6,Transformed test set shape,"(40620, 53)"
7,Numeric features,6
8,Categorical features,14
9,Rows with missing values,95.5%


In [7]:
best_model = compare_models(include=["xgboost","lightgbm"], sort = "RMSE")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,33187.6507,2889674815.1117,53747.9164,0.6133,0.8946,1.3560,6.5820
xgboost,Extreme Gradient Boosting,35582.8972,3343376102.7483,57780.0655,0.5521,0.9350,1.4633,7.4980


Processing:   0%|          | 0/13 [00:00<?, ?it/s]

Otrzymujemy RMSE na poziomie 53000 zobaczmy jaki wynik otrzymamy po przekształceniach danych

# Preprocessing

In [8]:
df["Data_pierwszej_rejestracji"] = pd.to_datetime(df["Data_pierwszej_rejestracji"],format="mixed",  errors="coerce")
df["Data_publikacji_oferty"] = pd.to_datetime(df["Data_publikacji_oferty"],format="mixed", errors="coerce")

Próbuje zamienić kolumny z typu object na date, daty są w kolumnie różnego typu stąd format="mixed", żeby dostosowywał się per przypadek. W kolumnie również są wartości nieprawidłowe jak 20/06/0012 w takim przypadku są wstawiane wartości null

In [9]:
def get_province(row):

    provinces = ['Dolnośląskie', 'Kujawsko-Pomorskie', 'Lubelskie', 'Lubuskie', 'Łódzkie', 'Małopolskie',
                   'Mazowieckie', 'Opolskie', 'Podkarpackie', 'Podlaskie', 'Pomorskie', 'Śląskie',
                   'Świętokrzyskie', 'Warmińsko-Mazurskie', 'Wielkopolskie', 'Zachodniopomorskie']


    for province in provinces:
        if province.lower() in str(row).lower():
            return province

    return None

df['Województwo_oferty'] = df['Lokalizacja_oferty'].apply(get_province)

Z szczegółowej lokalizacji oferty wyciągana jest informacja o województwie

In [10]:
def convert_to_list(s):
    try:
        return ast.literal_eval(s)
    except:
        return []

df['Wyposazenie_list'] = df['Wyposazenie'].apply(convert_to_list)
all_values = []
for lista in df['Wyposazenie_list']:
    if isinstance(lista, list):
        lowercase_values = [str(value).lower() for value in lista]
        all_values.extend(lowercase_values)

count_dict = Counter(all_values)


sorted_dict = dict(sorted(count_dict.items(), key=lambda item: item[1], reverse=True))


print(sorted_dict)

{'abs': 118133, 'central locking': 117470, 'electric front windows': 115037, 'drivers airbag': 114554, 'power steering': 114405, 'passengers airbag': 113354, 'electrically adjustable mirrors': 109870, 'factory radio': 107505, 'immobilizer': 106948, 'on-board computer': 97929, 'esp(stabilization of the track)': 91393, 'alloy wheels': 91177, 'cd': 87848, 'asr (traction control)': 87680, 'multifunction steering wheel': 87522, 'automatic air conditioning': 85136, 'isofix': 84673, 'front side airbags': 82571, 'fog lights': 80760, 'electric rear windows': 79127, 'cruise control': 76940, 'heated side mirrors': 75877, 'air curtains': 74858, 'rear parking sensors': 74707, 'bluetooth': 73815, 'rain sensor': 68582, 'alarm': 67547, 'daytime running lights': 66789, 'twilight sensor': 64942, 'usb socket': 64864, 'aux socket': 62797, 'mp3': 60884, 'dual zone air conditioning': 55956, 'electrochromic rear view mirror': 55384, 'heated front seats': 54649, 'tinted windows': 51808, 'gps navigation': 4926

Wartości w kolumnie Wyposazenie są zamieniane na typ list oraz tworzony jest słownik z unikalnymi wartościami w tej kolumnie

In [11]:
for feature in list(sorted_dict.keys()):

    df[f"{feature}_flg"] = df["Wyposazenie_list"].apply(
        lambda x: 1 if any(item.lower() == feature for item in x) else 0
    )

Dla każdej unikalnej wartości z słownika powstaje flaga czy dany wiersz posiadane daną wartość za pomocą flagi

In [12]:
df.columns

Index(['Cena', 'Waluta', 'Stan', 'Marka_pojazdu', 'Model_pojazdu',
       'Wersja_pojazdu', 'Generacja_pojazdu', 'Rok_produkcji', 'Przebieg_km',
       'Moc_KM', 'Pojemnosc_cm3', 'Rodzaj_paliwa', 'Emisja_CO2', 'Naped',
       'Skrzynia_biegow', 'Typ_nadwozia', 'Liczba_drzwi', 'Kolor',
       'Kraj_pochodzenia', 'Pierwszy_wlasciciel', 'Data_pierwszej_rejestracji',
       'Data_publikacji_oferty', 'Lokalizacja_oferty', 'Wyposazenie',
       'Województwo_oferty', 'Wyposazenie_list', 'abs_flg',
       'central locking_flg', 'electric front windows_flg',
       'drivers airbag_flg', 'power steering_flg', 'passengers airbag_flg',
       'electrically adjustable mirrors_flg', 'factory radio_flg',
       'immobilizer_flg', 'on-board computer_flg',
       'esp(stabilization of the track)_flg', 'alloy wheels_flg', 'cd_flg',
       'asr (traction control)_flg', 'multifunction steering wheel_flg',
       'automatic air conditioning_flg', 'isofix_flg',
       'front side airbags_flg', 'fog lights_f

In [13]:
def remove_outliers(df,col_method_dict):
  df_clean = df.copy()
  multiplier=1.5
  for column,method in col_method_dict.items():
    Q1 = df_clean[column].quantile(0.25)
    Q3 = df_clean[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    if method=="lower":
          idx_to_drop = df_clean[df_clean[column]<=lower_bound].index
    elif method=="upper":
          idx_to_drop = df_clean[df_clean[column]>=upper_bound].index
    else:
         idx_to_drop = df_clean[(df_clean[column]>=upper_bound) & (df_clean[column]<=lower_bound)].index
    df_clean.drop(index=idx_to_drop, inplace=True)
  return df_clean

def fill_null(df,col_method_dict):
  df_clean = df.copy()
  imputed_values_dict = {}
  for column,method in col_method_dict.items():
    if method=="mode":
          mode = df_clean[column].mode()[0]
          df_clean[column] = df_clean[column].fillna(mode)
          imputed_values_dict = {column: mode}
    elif method=="mean":
          mean = df_clean[column].mean()
          df_clean[column] = df_clean[column].fillna(mean)
          imputed_values_dict = {column: mean}
    else:
         df_clean[column] = df_clean[column].fillna(method)
         imputed_values_dict = {column: method}

  return df_clean,imputed_values_dict



def preproces_dataframe(df):
  df_to_preproces = df.copy()
  waluta_index_to_drop = df_to_preproces.query("Waluta!='PLN'").index
  null_marka_idx = df_to_preproces[df_to_preproces['Marka_pojazdu'].isnull()].index
  null_rok_idx = df_to_preproces[df_to_preproces['Rok_produkcji'].isnull()].index
  null_przebieg_idx = df_to_preproces[df_to_preproces['Przebieg_km'].isnull()].index
  null_moc_idx = df_to_preproces[df_to_preproces['Moc_KM'].isnull()].index

  weird_doors_number_idx = df_to_preproces[(df_to_preproces['Liczba_drzwi'] < 2) | (df_to_preproces['Liczba_drzwi'] > 9)].index

  indices_to_drop = pd.Index([])
  for idx_set in [waluta_index_to_drop, weird_doors_number_idx,null_marka_idx,null_rok_idx,null_przebieg_idx,null_moc_idx]:
      indices_to_drop = indices_to_drop.union(idx_set)

  indices_to_drop = indices_to_drop.intersection(df_to_preproces.index)
  df_to_preproces.drop(index=indices_to_drop, inplace=True)
  df_to_preproces,imputed_values_dict = fill_null(df_to_preproces,
    col_method_dict={"Kraj_pochodzenia": "Nieznany",
      "Pierwszy_wlasciciel": "No", "Województwo_oferty": "mode"})
  return df_to_preproces,imputed_values_dict


Część wierszy z wartościami null oraz z dziwnymi wartościami w kolumnach jest usuwana dla 3 kolumn wartości null są imputowane modą lub wartością stałą. Zwracany jest również słownik, który posłuży dla zbioru testowego

In [14]:
print(f"Rows before transformations: {len(df)}")
df_clean,imputed_values_dict = preproces_dataframe(df)
print(f"Rows afte transformations: {len(df_clean)}")

Rows before transformations: 135397
Rows afte transformations: 118307


# Feature Enginnering

In [15]:
mapping_auto_groups = {
    'Renault': 'grupa renault',
    'Opel': 'grupa stellantis',
    'Ford': 'grupa ford',
    'Toyota': 'grupa toyota',
    'BMW': 'grupa bmw',
    'Audi': 'grupa volkswagen',
    'Volkswagen': 'grupa volkswagen',
    'Hyundai': 'grupa hyundai',
    'Kia': 'grupa hyundai',
    'Nissan': 'grupa nissan',
    'Dacia': 'grupa renault',
    'Mazda': 'inny',
    'Volvo': 'grupa volvo',
    'Daewoo': 'grupa gm',
    'Fiat': 'grupa stellantis',
    'Jaguar': 'grupa jaguar land rover',
    'Škoda': 'grupa volkswagen',
    'Mitsubishi': 'grupa nissan',
    'Peugeot': 'grupa stellantis',
    'Seat': 'grupa volkswagen',
    'Porsche': 'grupa volkswagen',
    'Honda': 'inny',
    'Jeep': 'grupa stellantis',
    None: 'nieokreślony',
    'Citroën': 'grupa stellantis',
    'Mercedes-Benz': 'grupa daimler',
    'Chevrolet': 'grupa gm',
    'Subaru': 'inny',
    'MINI': 'grupa bmw',
    'Alfa Romeo': 'grupa stellantis',
    'DS Automobiles': 'grupa stellantis',
    'Smart': 'grupa daimler',
    'Suzuki': 'inny',
    'Buick': 'grupa gm',
    'Ferrari': 'inny',
    'Saab': 'inny',
    'Land Rover': 'grupa jaguar land rover',
    'Lancia': 'grupa stellantis',
    'Cadillac': 'grupa gm',
    'Infiniti': 'grupa nissan',
    'Aixam': 'inny',
    'Chrysler': 'grupa stellantis',
    'SsangYong': 'inny',
    'Lexus': 'grupa toyota',
    'Dodge': 'grupa stellantis',
    'McLaren': 'inny',
    'Gaz': 'grupa gaz',
    'Lada': 'grupa lada',
    'Triumph': 'inny',
    'Lincoln': 'grupa ford',
    'Daihatsu': 'grupa toyota',
    'Pontiac': 'grupa gm',
    'Rover': 'inny',
    'Hummer': 'grupa gm',
    'Tesla': 'inny',
    'Chatenet': 'inny',
    'Wołga': 'grupa fso',
    'Microcar': 'inny',
    'Cupra': 'grupa volkswagen',
    'Maserati': 'grupa stellantis',
    'RAM': 'grupa stellantis',
    'Isuzu': 'inny',
    'MG': 'inny',
    'Warszawa': 'grupa fso',
    'Polonez': 'grupa fso',
    'Acura': 'grupa honda',
    'Moskwicz': 'inny',
    'Rolls-Royce': 'grupa bmw',
    'Ligier': 'inny',
    'Abarth': 'grupa stellantis',
    'Inny': 'inny',
    'Bentley': 'grupa volkswagen',
    'Trabant': 'inny',
    'Austin': 'inny',
    'Mercury': 'grupa ford',
    'DFSK': 'inny',
    'Aston Martin': 'inny',
    'GMC': 'grupa gm',
    'DKW': 'inny',
    'Tarpan': 'inny',
    'Uaz': 'grupa uaz',
    'FAW': 'grupa faw',
    'Lotus': 'inny',
    'Plymouth': 'grupa stellantis',
    'Tata': 'grupa tata motors',
    'Żuk': 'grupa fso',
    'Zastava': 'inny',
    'Alpine': 'grupa renault',
    'Tavria': 'inny',
    'Casalini': 'inny',
    'Oldsmobile': 'grupa gm',
    'Iveco': 'grupa iveco',
    'Lamborghini': 'grupa volkswagen',
    'Syrena': 'grupa fso',
    'Vauxhall': 'grupa stellantis',
    'Wartburg': 'inny',
    'Grecav': 'inny',
    'Baic': 'inny',
    'Autobianchi': 'grupa stellantis',
    'Nysa': 'grupa fso',
    'Vanderhall': 'inny',
    'Zaporożec': 'inny',
    'Saturn': 'grupa gm'
}

brand_segment = {
    'Renault': 'mid-range',
    'Opel': 'mid-range',
    'Ford': 'mid-range',
    'Toyota': 'mid-range',
    'BMW': 'premium',
    'Audi': 'premium',
    'Volkswagen': 'mid-range',
    'Hyundai': 'mid-range',
    'Kia': 'mid-range',
    'Nissan': 'mid-range',
    'Dacia': 'budget',
    'Mazda': 'mid-range',
    'Volvo': 'premium',
    'Daewoo': 'budget',
    'Fiat': 'mid-range',
    'Jaguar': 'premium',
    'Škoda': 'mid-range',
    'Mitsubishi': 'mid-range',
    'Peugeot': 'mid-range',
    'Seat': 'mid-range',
    'Porsche': 'premium',
    'Honda': 'mid-range',
    'Jeep': 'mid-range',
    None: 'nieokreślony',
    'Citroën': 'mid-range',
    'Mercedes-Benz': 'premium',
    'Chevrolet': 'mid-range',
    'Subaru': 'mid-range',
    'MINI': 'premium',
    'Alfa Romeo': 'premium',
    'DS Automobiles': 'premium',
    'Smart': 'budget',
    'Suzuki': 'budget',
    'Buick': 'mid-range',
    'Ferrari': 'premium',
    'Saab': 'mid-range',
    'Land Rover': 'premium',
    'Lancia': 'mid-range',
    'Cadillac': 'premium',
    'Infiniti': 'premium',
    'Aixam': 'budget',
    'Chrysler': 'mid-range',
    'SsangYong': 'budget',
    'Lexus': 'premium',
    'Dodge': 'mid-range',
    'McLaren': 'premium',
    'Gaz': 'budget',
    'Lada': 'budget',
    'Triumph': 'mid-range',
    'Lincoln': 'premium',
    'Daihatsu': 'budget',
    'Pontiac': 'mid-range',
    'Rover': 'mid-range',
    'Hummer': 'mid-range',
    'Tesla': 'premium',
    'Chatenet': 'budget',
    'Wołga': 'budget',
    'Microcar': 'budget',
    'Cupra': 'premium',
    'Maserati': 'premium',
    'RAM': 'mid-range',
    'Isuzu': 'mid-range',
    'MG': 'budget',
    'Warszawa': 'budget',
    'Polonez': 'budget',
    'Acura': 'premium',
    'Moskwicz': 'budget',
    'Rolls-Royce': 'premium',
    'Ligier': 'budget',
    'Abarth': 'mid-range',
    'Inny': 'nieokreślony',
    'Bentley': 'premium',
    'Trabant': 'budget',
    'Austin': 'mid-range',
    'Mercury': 'mid-range',
    'DFSK': 'budget',
    'Aston Martin': 'premium',
    'GMC': 'mid-range',
    'DKW': 'budget',
    'Tarpan': 'budget',
    'Uaz': 'budget',
    'FAW': 'budget',
    'Lotus': 'premium',
    'Plymouth': 'mid-range',
    'Tata': 'budget',
    'Żuk': 'budget',
    'Zastava': 'budget',
    'Alpine': 'premium',
    'Tavria': 'budget',
    'Casalini': 'mid-range',
    'Oldsmobile': 'mid-range',
    'Iveco': 'mid-range',
    'Lamborghini': 'premium',
    'Syrena': 'budget',
    'Vauxhall': 'mid-range',
    'Wartburg': 'budget',
    'Grecav': 'budget',
    'Baic': 'budget',
    'Autobianchi': 'mid-range',
    'Nysa': 'budget',
    'Vanderhall': 'premium',
    'Zaporożec': 'budget',
    'Saturn': 'mid-range'
}



Dla unikatowych wartości w kolumnie marka powstaną dwie kolumny oznaczające przynależność do danej grupy oraz segment

In [16]:
df_clean["Rok_Pierwszej_Rejestracji"] = df_clean["Data_pierwszej_rejestracji"].dt.year
df_clean["Rok_publikacji_oferty"] = df_clean["Data_publikacji_oferty"].dt.year
df_clean["zajerestrowany_flg"] = df_clean["Data_pierwszej_rejestracji"].apply(lambda x: 1 if pd.notnull(x) else 0)
df_clean["Marka_pojazdu_group"] = df_clean["Marka_pojazdu"].map(mapping_auto_groups)
df_clean["Marka_pojazdu_segment"] = df_clean["Marka_pojazdu"].map(brand_segment)

In [17]:
df_model = df_clean.drop(columns=["Data_pierwszej_rejestracji","Data_publikacji_oferty","Lokalizacja_oferty","Wyposazenie","Wyposazenie_list","Waluta"])

In [18]:
df_model.reset_index(inplace=True, drop=True)

In [19]:
X = df_model.drop(columns=["Cena"])
y = df_model["Cena"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)

In [20]:
X_train.shape

(106476, 94)

In [21]:
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
X_val.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)
y_val.reset_index(inplace=True, drop=True)
categorical_columns = df_model.select_dtypes(include=['object']).columns
X_train_non_categorical = X_train[[col for col in X.columns if col not in categorical_columns]]
X_test_non_categorical = X_test[[col for col in X.columns if col not in categorical_columns]]
X_val_non_categorical =X_val[[col for col in X.columns if col not in categorical_columns]]
enc = TargetEncoder(target_type="continuous", cv=5, shuffle=True, random_state=42)
X_train_columns_encoded = enc.fit_transform(X_train[categorical_columns], y_train)
X_test_columns_encoded = enc.transform(X_test[categorical_columns])
X_val_columns_encoded = enc.transform(X_val[categorical_columns])
X_train_encoded = pd.concat([X_train_non_categorical, pd.DataFrame(X_train_columns_encoded,columns=categorical_columns)], axis=1)
X_test_encoded = pd.concat([X_test_non_categorical, pd.DataFrame(X_test_columns_encoded,columns=categorical_columns)], axis=1)
X_val_encoded = pd.concat([X_val_non_categorical, pd.DataFrame(X_val_columns_encoded,columns=categorical_columns)], axis=1)

In [ ]:
def objective(trial: optuna.Trial) -> float:

    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'random_state': 42,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
    }


    model = lgb.LGBMRegressor(**param)
    model.fit(X_train_encoded, y_train)


    y_pred = model.predict(X_val_encoded)

    rmse = mean_squared_error(y_val, y_pred,squared=False)


    return rmse



In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, show_progress_bar=True)

  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-03-25 19:04:06,472] Trial 0 finished with value: 19934.425553585384 and parameters: {'learning_rate': 0.03362944094621378, 'n_estimators': 797, 'max_depth': 11, 'num_leaves': 93, 'min_child_samples': 79, 'subsample': 0.9220158614325372, 'colsample_bytree': 0.8095186333302333, 'reg_alpha': 0.18336040701222117, 'reg_lambda': 6.528000400681945}. Best is trial 0 with value: 19934.425553585384.
[I 2025-03-25 19:04:28,475] Trial 1 finished with value: 20475.690261382228 and parameters: {'learning_rate': 0.04948301067224409, 'n_estimators': 914, 'max_depth': 8, 'num_leaves': 19, 'min_child_samples': 100, 'subsample': 0.8074244707823887, 'colsample_bytree': 0.6852677474515261, 'reg_alpha': 0.00017459896512289015, 'reg_lambda': 8.600906129524269}. Best is trial 0 with value: 19934.425553585384.
[I 2025-03-25 19:04:52,249] Trial 2 finished with value: 19962.291068190636 and parameters: {'learning_rate': 0.07502654870188795, 'n_estimators': 422, 'max_depth': 10, 'num_leaves': 41, 'min_chi

In [ ]:
best_params_ = study.best_params.copy()

In [ ]:
best_params_

{'learning_rate': 0.049364348819874115,
 'n_estimators': 921,
 'max_depth': 8,
 'num_leaves': 78,
 'min_child_samples': 5,
 'subsample': 0.6660114113089972,
 'colsample_bytree': 0.6592732651635046,
 'reg_alpha': 6.373661478478824,
 'reg_lambda': 1.5377330406104232e-05,
 'verbosity': -1,
 'random_state': 42}

In [24]:
model = lgb.LGBMRegressor(**best_params_)
model.fit(X_train_encoded, y_train)

LGBMRegressor(colsample_bytree=0.6592732651635046,
              learning_rate=0.049364348819874115, max_depth=8,
              min_child_samples=5, n_estimators=921, num_leaves=78,
              random_state=42, reg_alpha=6.373661478478824,
              reg_lambda=1.5377330406104232e-05, subsample=0.6660114113089972,
              verbosity=-1)

In [25]:
test_importance = permutation_importance(
            model, X_val_encoded, y_val,
            n_repeats=5, random_state=42, n_jobs=-1, scoring="neg_root_mean_squared_error"
        )

In [34]:
importance_df = pd.DataFrame({
            'feature': X_test.columns,
            'test_importance': test_importance.importances_mean
        })

In [35]:
importance_df.sort_values(by="test_importance", ascending=False)

,feature,test_importance
2,Model_pojazdu,26281.087596
0,Stan,23565.144745
1,Marka_pojazdu,12641.181246
3,Wersja_pojazdu,10327.284309
80,panoramic roof_flg,9666.642354
...,...,...
78,dvd player_flg,-67.236801
76,shift paddles_flg,-82.955247
77,active cruise control_flg,-83.614730
91,zajerestrowany_flg,-96.320247


Korzystając z permutation importance na zbiorze walidacyjnym najważniejsze cechy to model_pojazdu,stan,marka_pojazdu,wersja pojazdu oraz czy pojazd posiada dach panoramiczny

In [27]:

y_pred = model.predict(X_test_encoded)
rmse = mean_squared_error(y_test, y_pred,squared=False)
print(rmse)

22445.6177671327


In [28]:
y_pred = model.predict(X_val_encoded)
rmse = mean_squared_error(y_val, y_pred,squared=False)
print(rmse)

28115.12279247107


In [29]:
y_pred = model.predict(X_train_encoded)
rmse = mean_squared_error(y_train, y_pred,squared=False)
print(rmse)

9020.619877088107


Względem benchmarku udało się zminimalizować błąd +/- 30 tysięcy. Przy większej ilości czasu warto by było dokonać jeszcze selekcji zmiennych. Głównym wyzwaniem było odpowiednie przekształcenie danych przez różne formaty dat oraz dziwny zapis wyposażenia dla rekordu

In [30]:
sample_sub = pd.read_csv("sample_submission.csv")
df_test = pd.read_csv("/content/sales_ads_test.csv",index_col=0)

In [31]:
sample_sub

,ID,Cena
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
72902,72903,0
72903,72904,0
72904,72905,0
72905,72906,0


In [32]:
def transform_to_model(df_to_transform):
  df = df_to_transform.copy()
  df["Data_pierwszej_rejestracji"] = pd.to_datetime(df["Data_pierwszej_rejestracji"],format="mixed",  errors="coerce")
  df["Data_publikacji_oferty"] = pd.to_datetime(df["Data_publikacji_oferty"],format="mixed", errors="coerce")
  df['Województwo_oferty'] = df['Lokalizacja_oferty'].apply(get_province)
  df['Wyposazenie_list'] = df['Wyposazenie'].apply(convert_to_list)
  for feature in list(sorted_dict.keys()):

    df[f"{feature}_flg"] = df["Wyposazenie_list"].apply(
        lambda x: 1 if any(item.lower() == feature for item in x) else 0
    )
  for column,value in imputed_values_dict.items():
    df[column] = df[column].fillna(value)
  df["Rok_Pierwszej_Rejestracji"] = df["Data_pierwszej_rejestracji"].dt.year
  df["Rok_publikacji_oferty"] = df["Data_publikacji_oferty"].dt.year
  df["zajerestrowany_flg"] = df["Data_pierwszej_rejestracji"].apply(lambda x: 1 if pd.notnull(x) else 0)
  df["Marka_pojazdu_group"] = df["Marka_pojazdu"].map(mapping_auto_groups)
  df["Marka_pojazdu_segment"] = df["Marka_pojazdu"].map(brand_segment)
  df.drop(columns=["Data_pierwszej_rejestracji","Data_publikacji_oferty","Lokalizacja_oferty","Wyposazenie","Wyposazenie_list","Waluta"], inplace=True)
  df.reset_index(inplace=True, drop=True)
  categorical_columns = df.select_dtypes(include=['object']).columns
  df_non_categorical = df[[col for col in df.columns if col not in categorical_columns]]
  categorical_columns_encoded = enc.transform(df[categorical_columns])
  df_model_encoded = pd.concat([df_non_categorical, pd.DataFrame(categorical_columns_encoded,columns=categorical_columns)], axis=1)
  return df_model_encoded

In [33]:
X_scaled = transform_to_model(df_test)

In [36]:
X_scaled

,Rok_produkcji,Przebieg_km,Moc_KM,Pojemnosc_cm3,Emisja_CO2,Liczba_drzwi,abs_flg,central locking_flg,electric front windows_flg,drivers airbag_flg,...,Rodzaj_paliwa,Naped,Skrzynia_biegow,Typ_nadwozia,Kolor,Kraj_pochodzenia,Pierwszy_wlasciciel,Województwo_oferty,Marka_pojazdu_group,Marka_pojazdu_segment
0,2021,1.0,163.0,1950.0,NaN,5.0,1,1,1,1,...,57386.612471,158386.585915,114552.516506,107512.552883,67275.775299,95151.750170,73588.810178,82687.844424,109204.684639,104723.423038
1,2008,202585.0,145.0,1999.0,184.0,5.0,1,1,1,1,...,64629.891301,43927.985505,33099.917161,48135.922062,46262.061977,62302.003090,62302.003090,48392.723602,47309.624150,45384.983737
2,2005,373000.0,218.0,2993.0,NaN,5.0,1,1,1,1,...,57386.612471,71422.118752,114552.516506,76565.118661,72938.488798,62302.003090,62302.003090,57782.311610,99079.659516,104723.423038
3,2021,10.0,130.0,1498.0,NaN,5.0,1,1,1,1,...,64629.891301,43927.985505,33099.917161,40586.094508,78489.148559,62302.003090,62302.003090,53108.623901,69404.294970,45384.983737
4,2014,150000.0,245.0,2967.0,156.0,5.0,1,1,1,1,...,57386.612471,158386.585915,114552.516506,48135.922062,72938.488798,34115.111792,62302.003090,82687.844424,69404.294970,104723.423038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72902,2016,53000.0,220.0,1984.0,180.0,5.0,1,1,1,1,...,64629.891301,132231.411207,114552.516506,107512.552883,67275.775299,95151.750170,73588.810178,82687.844424,69404.294970,45384.983737
72903,2009,188132.0,136.0,1997.0,NaN,5.0,1,1,1,1,...,57386.612471,132231.411207,33099.917161,107512.552883,67275.775299,34115.111792,62302.003090,82687.844424,47309.624150,45384.983737
72904,2016,76786.0,143.0,1995.0,NaN,5.0,1,1,1,1,...,57386.612471,43927.985505,33099.917161,48135.922062,59860.753854,95151.750170,73588.810178,82687.844424,54023.231344,45384.983737
72905,2009,204300.0,114.0,1995.0,203.0,4.0,1,1,1,1,...,57386.612471,43927.985505,33099.917161,41222.660910,59860.753854,62302.003090,62302.003090,53879.473502,35508.565262,45384.983737


In [37]:
y_test_pred = model.predict(X_scaled)

In [38]:
y_test_pred = pd.DataFrame({"ID": list(df_test.index), "Cena":y_test_pred})

In [39]:
y_test_pred

,ID,Cena
0,1,197962.988241
1,2,18553.802097
2,3,21862.972261
3,4,103267.020857
4,5,88713.030077
...,...,...
72902,72903,112305.656285
72903,72904,29914.824520
72904,72905,61470.809748
72905,72906,26778.214334


In [40]:
y_test_pred.to_csv("submission.csv",index=False)